### calculate the SA signal for each order

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii, fits
from matplotlib.gridspec import GridSpec
from scipy import interpolate
from astropy.visualization import MinMaxInterval, AsinhStretch, HistEqStretch, ImageNormalize
from scipy.optimize import curve_fit
%matplotlib inline
#%matplotlib ipympl

In [26]:
# define the input/output parameters
path = '/Volumes/JPW_4TB/iSHELL/191008/'            # path to the directory containing the reduced and cal files

source = 'DRTau_PA90'
calfile = 'wavecal1-28'                             # associated wavecal file

source = 'HR1473_PA90'
calfile = 'wavecal34-45'

source = 'HR1473_PA270'
calfile = 'wavecal56-67'

source = 'DRTau_PA270'
calfile = 'wavecal68-95'

source = 'HLTau_PA138'
calfile = 'wavecal106-115'

source = 'HLTau_PA318'
calfile = 'wavecal116-125'

source = 'HR1473_PA318'
calfile = 'wavecal136-155'

source = 'HR1473_PA138a'
calfile = 'wavecal161-180'

source = 'FZTau_PA138'
calfile = 'wavecal181-206'

source = 'FZTau_PA318'
calfile = 'wavecal217-242'

source = 'DFTau_PA318'
calfile = 'wavecal243-256'

source = 'DFTau_PA138'
calfile = 'wavecal262-275'

source = 'HR1473_PA138b'
calfile = 'wavecal281-286'

rectfile = source+'_rectified'                      # combined rectified file for all observations of source

In [27]:
# load atmospheric transmission file
hdu = fits.open('/Users/jpw/idl/Spextool/data/atran75000.fits')
tdata = hdu[0].data
atrans = interpolate.interp1d(tdata[0,:], tdata[1,:])
hdu.close()

In [28]:
hdu1 = fits.open(path+'reduced/'+rectfile+'.fits')
flux = hdu1[0].data
var = hdu1[1].data
rect_hd = hdu1[0].header
#flux_norm = ImageNormalize(flux, interval=MinMaxInterval(), stretch=HistEqStretch(flux))
flux_norm = ImageNormalize(flux, vmin=-5, vmax=5, stretch=AsinhStretch(0.3))
nslit, nwl = flux.shape
flux_fit = np.zeros(flux.shape) + np.nan

hdu2 = fits.open(path+'cal/'+calfile+'.fits')
wc_hd = hdu2[0].header
orders = wc_hd['ORDERS'].split(',')
norders = wc_hd['NORDERS']

In [29]:
j1 = 30           # starting point of first order -- figured out by hand
dj_AB = 121       # width of each order -- this should match the size of the number of rows in the order extension
dj_blank = 30     # gap between orders -- this is figured out by eye and assumed to be the same for all orders

In [30]:
# two gaussians with different peaks and offsets but same FWHM
# added an extra constant offset since some orders are not perfectly sky-subtracted
def gauss2(x, A1, x1, A2, x2, fwhm, C):
    sigma = fwhm / (8*np.log(2))
    y = A1 * np.exp(-0.5*((x-x1)/sigma)**2) + A2 * np.exp(-0.5*((x-x2)/sigma)**2) + C
    return y

In [31]:
def fit_median(order_flux, pix_nofit=0, plotfile=None):
    # stack an order in wavelength to measure the average slit profile and guide individual fits
    im_median = np.nanmedian(order_flux, axis=1)
    j = np.arange(im_median.size)
    p0 = [np.min(im_median), np.argmin(im_median), np.max(im_median), np.argmax(im_median), 2, 0]

    # don't fit within s_nofit of the edges
    fit_range = (j > pix_nofit) & (j < j[-1]-pix_nofit)

    # fit two gaussians to the data
    pfit, pcov = curve_fit(gauss2, j[fit_range], im_median[fit_range], p0)
    #perr = np.sqrt(np.diag(pcov))

    if plotfile is not None:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.step(j, im_median)
        ax.plot(j[fit_range], gauss2(j[fit_range], *pfit))
        ax.set_xlabel(r"Row (pixels)", fontsize=14)
        ax.set_ylabel(r"Flux (Jy)", fontsize=14)
        fig.tight_layout()
        fig.savefig(plotfile, dpi=300)

    return pfit

In [32]:
def calculate_SA(order_flux, pix_nofit=0, nfwhm=1, verbose=False):
    ny, nx = order_flux.shape
    pfit = fit_median(order_flux, pix_nofit=pix_nofit)#, plotfile='median_profile.png')

    jneg =  pfit[1] + np.array([0, -0.5*nfwhm, 0.5*nfwhm]) * pfit[4]
    jpos =  pfit[3] + np.array([0, -0.5*nfwhm, 0.5*nfwhm]) * pfit[4]
    SA  = np.zeros((4, nx)) + np.nan

    for i in range(nx):
        flux_slice = order_flux[:, i]
        #weights = 1 / order_var[:, i]

        jsum1 = 0.
        jsum2 = 0.
        fsum = 0.
        for j1 in range(int(jneg[1]+0.5), int(jneg[2]+1.5)):
            f1 = flux_slice[j1]
            jsum1 += (j1-jneg[0]) * f1
            jsum2 += (j1-jpos[0])**2 * f1
            fsum += f1
        jmean = jsum1 / fsum
        SA[0, i] = jmean
        SA[2, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jneg[2] - jneg[1] + 1))

        jsum1 = 0.
        jsum2 = 0.
        fsum = 0.
        for j1 in range(int(jpos[1]+0.5), int(jpos[2]+1.5)):
            f1 = flux_slice[j1]
            jsum1 += (j1-jpos[0]) * f1
            jsum2 += (j1-jneg[0])**2 * f1
            fsum += f1
        jmean = jsum1 / fsum
        SA[1, i] = jmean
        SA[3, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jpos[2] - jpos[1] + 1))

    return SA

In [33]:
# loop through all orders and fit the profile for each wavelength, writing results to a csv file
# go through orders in reverse order so that the wavelength monotonically increases
csvfile = open(path+'reduced/'+source+'_SA.csv', 'w')
csvfile.write('wavelength,  off_neg, off_pos, err_neg, err_pos\n')

for n, order in enumerate(orders[::-1]):
    norder = len(orders) - n - 1
    j0 = j1 + (dj_AB + dj_blank) * norder

    wavecal = hdu2[3+norder].data
    wl0 = wavecal[0, 0, 1:]
    # pad the wavelength file for order 99 where the order cuts off before the end
    wl = np.pad(wl0, (0, nwl-wl0.size), 'edge')
    slitpos = wavecal[0, 1:, 0]
    print(f'Order = {order}, Min/Max wavelength = {wl.min()}, {wl.max()}')

    # set to True to get a lot of verbiage on fitting errors
    SA = calculate_SA(flux[j0:j0+dj_AB, :], pix_nofit=5, nfwhm=1)

    # flag out below a given atmospheric transmission
    flag = atrans(wl) < 0.5
    SA[:, flag] = np.nan

    for i in range(wl.size):
        csvfile.write(f'{wl[i]:11.9f}, {SA[0, i]:7.4f}, {SA[1, i]:7.4f}, {SA[2, i]:7.4f}, {SA[3, i]:7.4f}\n')

csvfile.close()

Order = 114, Min/Max wavelength = 4.509236079287453, 4.547328436242248
Order = 113, Min/Max wavelength = 4.548995116940285, 4.587427849125159
Order = 112, Min/Max wavelength = 4.589464780224784, 4.628247118630138
Order = 111, Min/Max wavelength = 4.630664275248911, 4.669805700351494


/var/folders/tx/plmj8k8969gcvbqj4tml5hvm0000gn/T/ipykernel_67596/2368970709.py:35: RuntimeWarning: invalid value encountered in sqrt
  SA[3, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jpos[2] - jpos[1] + 1))
/var/folders/tx/plmj8k8969gcvbqj4tml5hvm0000gn/T/ipykernel_67596/2368970709.py:23: RuntimeWarning: invalid value encountered in sqrt
  SA[2, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jneg[2] - jneg[1] + 1))


Order = 110, Min/Max wavelength = 4.672613506524913, 4.712123757360066
Order = 109, Min/Max wavelength = 4.715333109006233, 4.755222192656282
Order = 108, Min/Max wavelength = 4.75884448190425, 4.799122683426173
Order = 107, Min/Max wavelength = 4.8031698244012855, 4.843847717218301
Order = 106, Min/Max wavelength = 4.848332173385103, 4.889420630168413
Order = 105, Min/Max wavelength = 4.894355443339662, 4.935865647408387
Order = 104, Min/Max wavelength = 4.941264468537273, 4.983207925806451
Order = 103, Min/Max wavelength = 4.98908504768853, 5.031473599197125
Order = 102, Min/Max wavelength = 5.037843991218735, 5.080689826271753
Order = 101, Min/Max wavelength = 5.087569171352807, 5.130884841314025
Order = 100, Min/Max wavelength = 5.138265947715741, 5.182088007979612
Order = 99, Min/Max wavelength = 5.190011499179449, 5.233667791109747
